# Weekly Data Pull from First Map and Federal Agencies

<b>Notes:</b>

<ul> 
    <li>Use the text document located on TFS under Mapping/Python/Python2/ESRI-TASK-SVR; Data_FM2 contains REST points for First Map 2.0 Services and for the federal government services that we consume. Data_Source_List contains First Map 1.0 REST points and the federal government services that we consume.</li>
    <li>The REST points can be queried with a variety of tools including python packages like requests and ArcGIS API for Python. Also, by using the GUI in ArcGIS Pro (I recommend Pro due to its ease of use when adding REST Servers).</li>
</ul>

<b>TO DO:</b>
<ul>
<li>Create Variables for all the REST Servers.</li>
<li>Add to weekly script</li>
</ul>

## First Map Data Processing

<b> Import Modules </b>

In [ ]:
import pandas as pd
import arcpy as arc
from arcpy import env
from arcgis.gis import GIS
from arcgis import features
import numpy as np
from datetime import datetime as dt
from pathlib import Path
import requests
from urllib.parse import urljoin
# import geopandas as gpd

### Connect to KC Portal to access Enterprise, KC AGOL site, and KC Staging.

In [ ]:
from arcgis.gis.server import Server
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
pURL = os.getenv('PORTAL_SITE')
aURL = os.getenv('AGOL_SITE')
sURL = os.getenv('STAGE_SITE')
sUser = os.getenv('STAGE_USERNAME')
sPass = os.getenv('STAGE_PASSWORD')
aUser = os.getenv('AGOL_USERNAME')
aPass = os.getenv('AGOL_PASSWORD')
pUser = os.getenv('ESRI_USERNAME')
pPass = os.getenv('ESRI_PASSWORD')

#Connect to Enterprise GIS
gisE = GIS(url=pURL, username=pUser, password=pPass)

#Connect to AGOL
gisA = GIS(url=aURL, username=aUser, password=aPass, set_active=False)

#Connect to Staging. Does not work for right now. Found a solution: https://community.esri.com/t5/developers-questions/error-connecting-to-arcgis-server-with-arcgis-python-api/m-p/869042?commentID=896496#comment-896496

# gisT = GIS(url=sURL, username=sUser, password=sPass, set_active=False, verify_cert=False)
# from arcgis.gis.server import Server
# sURL = 'https://ESRISTAGE:6443'
# gisT = Server(url=f'{sURL}/ed8fb7ce-e99d-4eb2-af78-b7ef11e3c2c4/admin',token_url=f'{sURL}/ed8fb7ce-e99d-4eb2-af78-b7ef11e3c2c4/tokens/generateToken', username=sUser, password=sPass, verify_cert=False, set_active=False)

#Must use Server Module for now
gisT = Server(url=f'{sURL}/arcgis/admin', username=sUser, password=sPass, verify_cert=False)

# gisT = GIS(url=f'{sURL}/arcgis/admin', username=sUser, password=sPass, verify_cert=False)

### Create Folders in the Users Directory since it should have r/w permissions for the user.

In [ ]:
# arc.env.overwriteOutput = True
# arc.env.qualifiedFieldNames = False
now = dt.today()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
wStr = now.strftime('%U')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'

### Only use with notebook

In [ ]:
pd.options.display.max_columns = 40

### Create File GeoDatabase and Feature Datasets

In [ ]:
#Create Folders for Permits Data
wFolder = [f for f in lPath if f.name == 'Processing'][0]
wProcessing = wFolder / 'Weekly' / f'{wStr}'
if wProcessing.exists() == True:
    print(f'{wProcessing} already exist.')
else:
    wProcessing.mkdir(parents=True)
    print(f'Created {wProcessing}.')

wFR = [f for f in lPath if f.name == 'Review'][0]
wReview = wFR / 'Weekly' / f'{wStr}'
if wReview.exists() == True:
    print(f'{wReview} already exist')
else:
    wReview.mkdir(parents=True)
    print(f'Created {wReview}')

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / wFolder / f'Data_{mStr}.gdb'
locGDB = outGDB / f'Weekly_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{wFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {wFolder}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

### Create Variables for Feature Layers from First Map.

In [ ]:
# gis = GIS()
# sdeConn = Path('G:\\MasterGISFiles\\Ben\\GISPro\\Database Connection\\PROD-WA.sde')
# locGDB = Path('C:\\Users\\MKinnaman\\GIS\\Processing')
fMapURL = 'https://enterprise.firstmap.delaware.gov/arcgis/rest/services/'
req = requests.get(fMapURL)
fmFS = set()
fmMS = set()
fmMSL = []
fmFCL = []
if req.status_code == 200: #check to make sure the site is active
    jFM = urljoin(fMapURL, '?f=json')
    fmF = requests.get(jFM).json()['folders']
    fmList = [x for x in fmF]
    # print(fmList)
    fmLink = [urljoin(fMapURL, f'{f}') for f in fmList]
    for x in fmLink:
        if x.split('/')[-1] == 'Hydrology':
            # print(x)
            aUrl = urljoin(x, '?f=json')
            a = requests.get(aUrl).json()
            # print(a)            
            for b in a['services']:
                if b['type'] == 'FeatureServer':
                    fmFS.add(b.get('name'))
                else:
                    fmMS.add(b.get('name'))
            d = fmMS - fmFS
            for c in d:
                a = urljoin(x, f'{c}/''MapServer/')
                flList = features.FeatureLayerCollection(a)
                for e in flList.layers:
                    fmMSL.append(e)
            for e in fmFS:
                a = urljoin(x, f'{e}/''FeatureServer/')
                flList = features.FeatureLayerCollection(a)
                for f in flList.layers:
                    fmFCL.append(f)
else:
    print(f'Could not connect to {fMapURL}')

Create URLs for Feature Collections on First Map

In [ ]:
gis = GIS()
# sdeConn = Path('G:\\MasterGISFiles\\Ben\\GISPro\\Database Connection\\PROD-WA.sde')
# locGDB = Path('C:\\Users\\MKinnaman\\GIS\\Processing')
fMapURL = 'https://enterprise.firstmap.delaware.gov/arcgis/rest/services/'
req = requests.get(fMapURL)

# def build(url, field):
#     return f'{urljoin(url, f"{field}")}'

# urls = [build(fMapURL, f) for f in sList]
sNames = []
sUrls = []
dNames = []
if req.status_code == 200:
    jFM = urljoin(fMapURL, '?f=json')
    fmF = requests.get(jFM).json()['folders']
    fmList = [x for x in fmF]
    # print(fmList)
    fmLink = [urljoin(fMapURL, f'{f}') for f in fmList]
    for x in fmLink:
        if x.split('/')[-1] == 'Hydrology':
            # print(x)
            aUrl = urljoin(x, '?f=json')
            a = requests.get(aUrl).json()
            # print(a)
            for b in a['services']:
                if b.get('type') != "MapServer":
                    sNames.append(b.get('name'))
                elif b.get('name') == 'Hydrology/DE_DFIRM':
                    sNames.append(b.get('name'))

        if x.split('/')[-1] == 'Society':
            # print(x)
            aUrl = urljoin(x, '?f=json')
            a = requests.get(aUrl).json()
            # print(a)
            for b in a['services']:
                if b.get('type') != "MapServer":
                    sNames.append(b.get('name'))
                    
        if x.split('/')[-1] == 'Boundaries':
            # print(x)
            aUrl = urljoin(x, '?f=json')
            a = requests.get(aUrl).json()
            # print(a)
            for b in a['services']:
                if b.get('type') != "MapServer":
                    sNames.append(b.get('name'))

        if x.split('/')[-1] == 'Geology':
            # print(x)
            aUrl = urljoin(x, '?f=json')
            a = requests.get(aUrl).json()
            # print(a)
            for b in a['services']:
                if b.get('type') != "MapServer":
                    sNames.append(b.get('name'))        

    for x in sNames:
        d = f'{x}/FeatureServer'
        sUrls.append(urljoin(fMapURL, d))

else:
    print(f'Could not connect to {fMapURL}')

In [ ]:
bflc = features.FeatureLayerCollection(sUrls[0]).layers
exp = "NAME = 'Kent'"
kcboun = bflc[4].query(where=exp, out_sr=26957, as_df=True) #KC boundary to try to clip the first map services

In [15]:
ae = kcboun.spatial.full_extent

In [ ]:
#Create geopandas DF to clip the services if the other one fails
import geopandas as gpd
kcboun_gf = gpd.GeoDataFrame(kcboun, geometry='SHAPE', crs=26957)
cl_gf = 

In [13]:
ac = features.FeatureLayerCollection(sUrls[4]).layers
# sen = locGDB / f'Del_Senate_{dStr}'
sen = ac[0].query(out_sr=26957, as_df=True)
# ac[0].query(out_sr=26957).save(f'{sen.parent}', f'{sen.name}')

In [ ]:
lol = kcboun.spatial.overlay(sen, op='intersection')
lol

In [ ]:
ag.spatial.to_featureclass(f'{locGDB / "SenDist"}')

In [ ]:
from arcgis.features import GeoSeriesAccessor
ad = GeoSeriesAccessor(kcboun.SHAPE)
ext = ad.extent
ext
# gf = GeoSeriesAccessor(ed_df.SHAPE)
# ar = gf.clip(ext)
# ar

In [ ]:
flc = features.FeatureLayerCollection(sUrls[4]).layers

In [ ]:
flc.layers[0].properties.name

In [ ]:
# fl[0]
start = 0
ed_df = pd.DataFrame()
# stop = len(fl)
for i in flc:
    if i.properties.name == 'Election Districts_2012':
        df = i.query(as_df=True, out_sr=26957)
        ed_df = df.copy(deep=True)

ed_df
    # start = 0
    # stop = len(fl)
    # print(f'{f[i].properties.name}')
# fl.properties.name

In [ ]:
# flC = fmFCL[0]
# fl = flC.query(out_sr=26957)
# fl.save(f'{locGDB}', 'Test')
# print('Saved')

In [ ]:
fl.GNIS_ID.fillna('', inplace=True)
fl.GNIS_ID.isna().sum()

In [ ]:
fl.GNIS_NAME.fillna('', inplace=True)

In [ ]:
fl.FCODENAME.fillna('', inplace=True)
fl.FTYPENAME.fillna('', inplace=True)

In [ ]:
fName = f'{locGDB / f"FMBLines_{dStr}"}'
fl.spatial.to_featureclass(fName,sanitize_columns=False)

In [ ]:
fl.spatial.to_featureclass(f'{locGDB / f"FMRiver_{dStr}"}',sanitize_columns=False)

<b> Create Variables for Feature Layers from Federal Agencies. </b>

In [ ]:
# Write before and after counts to a text file for review later.

## Federal Agencies Data Processing

https://hazards.fema.gov/gis/nfhl/rest/services/FIRMette/NFHLREST_FIRMette/MapServer/2/query
CID IN ('100001', '100006', '100009', '100010', '100014', '100042', '100002', '240045', '100015', '100017', '100020')


https://hazards.fema.gov/gis/nfhl/rest/services/MPP/MPP_GIS/FeatureServer/25/query to get CID within Kent County use 'COUNTY_FIPS = "10001"'